# 1.Google Collab Environment Setup

Please run this Google Colab Jupyter Notebook with the following settings under "Runtime" -> "Change runtime type".

Runtime type: Python 3

Hardware accelerator: T4 GPU

## Setting up Python 3.8
This project uses Python 3.8 whilst Google Colab currently uses Python 3.10. Run the following commands to downgrade to Python 3.8.

Note: This process takes quite some time so you can leave it running in the background.

Install Required Dependencies

In [ ]:
# downgrade Python by reinstalling pip and distutils
!apt-get install python3.8 python3-pip python3.8-distutils
!update-alternatives --install /usr/local/bin/python3 python3 /usr/bin/python3.8 1

# check the result
!python --version

!apt-get update
!apt install software-properties-common
!sudo dpkg --remove --force-remove-reinstreq python3-pip python3-setuptools python3-wheel
!apt-get install python3-pip

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libpython3.8-minimal libpython3.8-stdlib mailcap mime-support
  python3-setuptools python3-wheel python3.8-lib2to3 python3.8-minimal
Suggested packages:
  python-setuptools-doc python3.8-venv binfmt-support
The following NEW packages will be installed:
  libpython3.8-minimal libpython3.8-stdlib mailcap mime-support python3-pip
  python3-setuptools python3-wheel python3.8 python3.8-distutils
  python3.8-lib2to3 python3.8-minimal
0 upgraded, 11 newly installed, 0 to remove and 18 not upgraded.
Need to get 7,094 kB of archives.
After this operation, 29.1 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 mailcap all 3.70+nmu1ubuntu1 [23.8 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 mime-support all 3.66 [3,696 B]
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 p

In [ ]:
# clone FollowYourPose repository
!git clone https://github.com/mayuelala/FollowYourPose.git

# clone team 15's repository
!git clone https://github.com/ong-yi-xuan/ict3104-team15.git

Cloning into 'FollowYourPose'...
remote: Enumerating objects: 325, done.
remote: Counting objects: 100% (88/88), done.
remote: Compressing objects: 100% (68/68), done.
remote: Total 325 (delta 35), reused 71 (delta 20), pack-reused 237
Receiving objects: 100% (325/325), 277.89 MiB | 36.71 MiB/s, done.
Resolving deltas: 100% (92/92), done.


In [33]:
# download txt2video file
txt2video_file_id = "19jT40m76ZPkF_zUVa66QGiBJP9UsSvjn"
url = 'https://drive.google.com/uc?id=' + txt2video_file_id
output = 'txt2video.py'  # Desired name of the downloaded file
gdown.download(url, output, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=19jT40m76ZPkF_zUVa66QGiBJP9UsSvjn
To: /content/txt2video.py
100%|██████████| 7.14k/7.14k [00:00<00:00, 5.44MB/s]


'txt2video.py'

In [ ]:
# install required dependencies
!python3 -m pip install -q $(grep -ivE "triton|torch|torchvision|xformers" FollowYourPose/requirements.txt) # Install all except those specified (outdated in the req file)
!python3 -m pip install -q triton==2.0.0 torch==2.0.1 torchvision imageio-ffmpeg xformers==0.0.21 --extra-index-url https://download.pytorch.org/whl/cu113
!python3 -m pip install ipywidgets
!python3 -m pip install gdown

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.5/126.5 KB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 43.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 813.6/813.6 KB 56.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 KB 15.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.6/80.6 KB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 KB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 53.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 MB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 KB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.6/96.6 KB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.0/300.0 KB 32.7 MB/s et

In [ ]:
# importing other dependencies
import yaml
import gdown
import os
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML, Image

# Getting Charades Videos

## Connect to Google Drive to access sample videos

To simply the video uploading process, you will have to mount your google drive to the notebook to prevent loss of data after the runtime has been discarded.

In [29]:
file_id = '1j-MtW4Omn2n5zKFjojDGugaUg3dhnBKF'
output_file = 'video.mp4'
download_url = f'https://drive.google.com/uc?id={file_id}'

gdown.download(download_url, output_file, quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1j-MtW4Omn2n5zKFjojDGugaUg3dhnBKF
To: /content/video.mp4
100%|██████████| 13.9M/13.9M [00:00<00:00, 68.0MB/s]


'video.mp4'

## Playing a video
After a video has been uploaded to the notebook, select one from the list of compatible video types to play them.


In [ ]:
# yixuan use this code to play the .gif output_display

# there's no code to create this folder since we're cloning FYP's directory structure
main_directory = 'FollowYourPose/gif_results'
folder_contents = os.listdir(main_directory)

folder_dropdown = widgets.Dropdown(
    options=folder_contents,
    description='Select a folder:',
)

gif_dropdown = widgets.Dropdown(
    options=['Select a folder first'],
    description='Select a .gif file:',
    disabled=True,
)

button = widgets.Button(description="Play .gif File")
output = widgets.Output()

# Function to update the .gif dropdown based on the selected folder
def update_gif_dropdown(change):
    selected_folder = change.new
    selected_directory = os.path.join(main_directory, selected_folder)

    if os.path.isdir(selected_directory):
        gif_files = [f for f in os.listdir(selected_directory) if f.lower().endswith('.gif')]
        gif_dropdown.options = gif_files
        gif_dropdown.disabled = False
    else:
        gif_dropdown.options = ['Select a folder first']
        gif_dropdown.disabled = True

# Function to play the selected .gif file
def play_gif_file(_):
    selected_gif = gif_dropdown.value
    selected_folder = folder_dropdown.value
    selected_directory = os.path.join(main_directory, selected_folder)
    save_path = os.path.join(selected_directory, selected_gif)

    if selected_gif.lower().endswith('.gif'):
        with output:
            clear_output(wait=True)
            display(Image(open(save_path, 'rb').read(), format='png'))
    else:
        with output:
            clear_output(wait=True)
            print("Please select a valid .gif file.")

folder_dropdown.observe(update_gif_dropdown, names='value')
button.on_click(play_gif_file)

display(folder_dropdown)
display(gif_dropdown)
display(button)
display(output)


Dropdown(description='Select a folder:', options=('a man in the park, Van Gogh style.gif', 'A Iron man on the …

Dropdown(description='Select a .gif file:', disabled=True, options=('Select a folder first',), value='Select a…

Button(description='Play .gif File', style=ButtonStyle())

Output()

# Inference

## 1. Create folder to store models

In [ ]:
# for danial: by right we'd need to have this directory alr to save pre-trained models so we can make the folder in our repo?

# create folder to store model
%mkdir checkpoints
!git lfs install

Git LFS initialized.


## 2. Download or select pre-trained model


In [ ]:
#@markdown Download FollowYourPose model from remote repo
MODEL_NAME = "YueMafighting/FollowYourPose_v1" #@param {type:"string"}

#@markdown  Untick checkbox below if the model is loaded from a local path
download_pretrained_model = True #@param {type:"boolean"}
if download_pretrained_model:
    SAVE_LOCATION = f"./checkpoints/"
    !git clone https://huggingface.co/$MODEL_NAME checkpoints/

# TODO @DANIAL:
# else:
  # ADD CODE FOR SELECTING FILE FROM OWN FOLDER IF HAVE

print(f"[*] saved successfully in {SAVE_LOCATION}")

Cloning into 'checkpoints'...
remote: Enumerating objects: 42, done.
remote: Total 42 (delta 0), reused 0 (delta 0), pack-reused 42
Unpacking objects: 100% (42/42), 584.62 KiB | 5.79 MiB/s, done.
Filtering content: 100% (9/9), 9.75 GiB | 45.70 MiB/s, done.
[*] saved successfully in ./checkpoints/


## 3. Set Inference Configuration Settings

The code below saves user defined configuration settings into a configuration file for running inference.
<br>
Important When using Google Colab, the maximum video length should be 8 due to lack of resources.

<br>

**To save the config file, change the inputs as desired and run the cell again**

In [27]:
#@markdown Inference: Config for generating videos
config_file_name = "pose_test.yaml" #@param {type:"string"}
pretrained_model_path = "./checkpoints/stable-diffusion-v1-4" #@param {type:"string"}
video_output_dir = "gif_results" #@param {type:"string"}

# TODO: CODE FOR ACCEPTING MULTIPLE PROMPTS AND CONVERT INTO EACH PROMPT ARRAY
prompts = "Iron man on the beach" #@param {type:"string"}
prompts_list = [x.strip() for x in prompts.split(',')]

# checkpoint config
resume_from_checkpoint = "./checkpoints/followyourpose_checkpoint-1000" #@param {type:"string"}

#@markdown <br> Default video settings
# seed for reproducibility
seed = 33 #@param {type:"integer"}

# video config settings
video_length = 8 #@param {type:"integer"}
video_width = 512 #@param {type:"integer"}
video_height = 512 #@param {type:"integer"}

# inference config settings
num_inference_steps = 50 #@param {type:"number"}
guidance_scale = 12.5 #@param {type:"number"}
use_inv_latent = False #@param {type:"boolean"}
num_inv_steps = 50 #@param {type:"number"}
dataset_set = "val" #@param {type:"string"}

train_batch_size = 1
validation_steps = 100
mixed_precision = 'no'
gradient_checkpointing = False
enable_xformers_memory_efficient_attention = True

# break down prompts into individual prompts by comma


# save as config file
config_dir = "FollowYourPose/configs"

# save each config item into dictionary
config_dict = {
  'pretrained_model_path': pretrained_model_path,
  'output_dir': video_output_dir,
  'validation_data':
    {
      'prompts': prompts_list,
      'video_length': video_length,
      'width': video_width,
      'height': video_height,
      'num_inference_steps': num_inference_steps,
      'guidance_scale': guidance_scale,
      'use_inv_latent': use_inv_latent,
      'num_inv_steps': num_inv_steps,
      'dataset_set': dataset_set,
    },
  'train_batch_size': train_batch_size,
  'validation_steps': validation_steps,
  'resume_from_checkpoint': resume_from_checkpoint,
  'seed': seed,
  'mixed_precision': mixed_precision,
  'gradient_checkpointing': gradient_checkpointing,
  'enable_xformers_memory_efficient_attention': enable_xformers_memory_efficient_attention
}

# save dictionary as config yaml file
config_file = config_dir + '/' + config_file_name
with open(config_file, 'w') as file:
  yaml.dump(config_dict, file, sort_keys=False)

print("Config file saved as " + config_file)


Config file saved as FollowYourPose/configs/pose_test.yaml


## 4. Generate Content

**To generate the inference outputs files, change the file paths as desired and run the cell again**

In [34]:
config_file = "FollowYourPose/configs/pose_test.yaml" #@param {type:"string"}
MMPOSE_skeleton_file = "FollowYourPose/pose_example/vis_ikun_pose2.mov" #@param {type:"string"}

!accelerate launch txt2video.py --config={config_file}  --skeleton_path={MMPOSE_skeleton_file}

The following values were not passed to `accelerate launch` and had defaults used instead:
	`--num_processes` was set to a value of `1`
	`--num_machines` was set to a value of `1`
	`--mixed_precision` was set to a value of `'no'`
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
/usr/local/lib/python3.8/dist-packages/torchvision/transforms/_functional_video.py:6: UserWarning: The 'torchvision.transforms._functional_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms.functional' module instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/transforms/_transforms_video.py:22: UserWarning: The 'torchvision.transforms._transforms_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms' module instead.
  warnings.warn(
gif_results
10/09/2023 16:38:41 -

## 5. Play Videos

Select the desired folder and play the generated videos

In [37]:
main_directory = 'gif_results'

# create textbox to get user inputted directory
select_dir_textbox = widgets.Text(
    value=main_directory,
    placeholder=main_directory,
    description='Input:',
    disabled=False
)

folder_contents = os.listdir(main_directory)

folder_dropdown = widgets.Dropdown(
    options=folder_contents,
    description='Select a folder:',
)

gif_dropdown = widgets.Dropdown(
    options=['Select a folder first'],
    description='Select a .gif file:',
    disabled=True,
)

button = widgets.Button(description="Play .gif File")
output = widgets.Output()

# get user inputted value from textbox into main_directory var
def select_directory(sender):
  main_directory = {select_dir_textbox.value}

# Function to update the .gif dropdown based on the selected folder
def update_gif_dropdown(change):
    selected_folder = change.new
    selected_directory = os.path.join(main_directory, selected_folder)

    if os.path.isdir(selected_directory):
        gif_files = [f for f in os.listdir(selected_directory) if f.lower().endswith('.gif')]
        gif_dropdown.options = gif_files
        gif_dropdown.disabled = False
    else:
        gif_dropdown.options = ['Select a folder first']
        gif_dropdown.disabled = True

# Function to play the selected .gif file
def play_gif_file(_):
    selected_gif = gif_dropdown.value
    selected_folder = folder_dropdown.value
    selected_directory = os.path.join(main_directory, selected_folder)
    save_path = os.path.join(selected_directory, selected_gif)

    if selected_gif.lower().endswith('.gif'):
        with output:
            clear_output(wait=True)
            display(Image(open(save_path, 'rb').read(), format='png'))
    else:
        with output:
            clear_output(wait=True)
            print("Please select a valid .gif file.")

# event handlers
select_dir_textbox.on_submit(select_directory)
folder_dropdown.observe(update_gif_dropdown, names='value')
button.on_click(play_gif_file)

# display all created UI elements
display(select_dir_textbox)
display(folder_dropdown)
display(gif_dropdown)
display(button)
display(output)


Text(value='gif_results', description='Input:', placeholder='gif_results')

Dropdown(description='Select a folder:', options=('config.yaml', 'inv_latents', 'samples', 'inference'), value…

Dropdown(description='Select a .gif file:', disabled=True, options=('Select a folder first',), value='Select a…

Button(description='Play .gif File', style=ButtonStyle())

Output()

## 6. Exit and free runtime memory

In [ ]:
exit()